# Token price API

In [ ]:
#| default_exp price

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import functools, asyncio
from dataclasses import dataclass
from typing import Tuple, List
from sugar.token import Token
from sugar.helpers import chunk

In [ ]:
#| export

@dataclass(frozen=True)
class Price:
    """Data class for Token Price

    based on:
    https://github.com/velodrome-finance/oracle/blob/main/contracts/VeloOracle.sol
    """

    token: Token
    price: float

    @property
    def pretty_price(self) -> float: return round(self.price, 5)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()